In [1]:
%reset
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import scipy.io
import scipy.stats
import mne
from mne.time_frequency import tfr_morlet
from mne.stats import permutation_cluster_1samp_test
import gc
import os
import copy
import pickle
from os.path import exists
import mne
import numpy as np
from mne import create_info
from IPython.utils import io
import yasa
import seaborn as sns
import pandas as pd
import matlab
import matlab.engine
import os

gc.collect()

30233

In [2]:
pkl_dir = "C:\\Users\\User\\Cloud-Drive\\BigFiles\\OmissionExpOutput\\\imported_eventDetectionChan\\filter0.1"
import_type = "eventDetectionChan"
output_dir_name = 'eventDetection\\done_on_imported'

In [3]:
import_path = f'{pkl_dir}\\{import_type}.pkl'

with open(import_path, "rb") as file:
    [allsubsdata_perFile, configu] = pickle.load(file)

fig_output_dir = f"{configu['outputs_dir_path']}\\{output_dir_name}"
if not os.path.exists(fig_output_dir):
    os.mkdir(fig_output_dir)

In [4]:
configu['electrodes_names']


array(['EMG1', 'EMG2', 'EMG3', 'LOC', 'ROC', 'Fp1', 'Fp2', 'C3', 'C4',
       'O1', 'O2', 'F3', 'F4', 'P3', 'P4', 'Cz'], dtype=object)

One file yasa detection 

In [5]:
# sub = '32'
# datafile = 2
# id = f"{sub}_{datafile}"

# datafile_data = allsubsdata_perFile[id]['data'] ## shape (electrode, time)
# datafile_scoring = allsubsdata_perFile[id]['scoring'] ## shape (time/sampling/30)

# exmp_scoring_upsampled = yasa.hypno_upsample_to_data(datafile_scoring, 1/30, datafile_data, sf_data=configu['sample_freq'], verbose=True)
# sp = yasa.spindles_detect(datafile_data,sf=configu['sample_freq'], hypno = exmp_scoring_upsampled,include=[2, 3],ch_names=configu['electrodes_names'],multi_only=True)
# print(sp)
# print(sp.summary())
# # sp.plot_average(center='Peak',time_before = 1,time_after = 1)
# # plt.show()
# #%matplotlib widget
# #sp.plot_detection()

In [6]:
# %matplotlib inline
# sp.plot_average(center='Peak', time_before=0.1, time_after=0.1);

Batch yasa detection and save

In [7]:
# yasa_output_dir = f"{configu['outputs_dir_path']}/{output_dir_name}/yasa"
# if not os.path.exists(yasa_output_dir):
#     os.mkdir(yasa_output_dir)

# for id in allsubsdata_perFile:
#     datafile_data = allsubsdata_perFile[id]['data'] ## shape (electrode, time)
#     datafile_scoring = allsubsdata_perFile[id]['scoring'] ## shape (time/sampling/30)
#     if 2 not in datafile_scoring or 3 not in datafile_scoring:
#         continue

#     exmp_scoring_upsampled = yasa.hypno_upsample_to_data(datafile_scoring, 1/30, datafile_data, sf_data=configu['sample_freq'], verbose=False)
#     sp = yasa.spindles_detect(datafile_data,sf=configu['sample_freq'], hypno = exmp_scoring_upsampled,include=[2, 3],ch_names=configu['electrodes_names'],multi_only=True)
#     #summary_df  = sp.summary()
#     summary_df.to_csv(f"{yasa_output_dir}\\{allsubsdata_perFile[id]['subnum']}_{allsubsdata_perFile[id]['filenum']}_yasaSpindles.csv")
#     sp.plot_average(center='Peak', time_before=1, time_after=1)
#     plt.savefig(f"{yasa_output_dir}\\{allsubsdata_perFile[id]['subnum']}_{allsubsdata_perFile[id]['filenum']}_yasaSpindles_average.png")

One file Nir&Andrillon Detection

In [8]:
# import matlab
# import matlab.engine

# sub = '32'
# datafile = 2 
# id = f"{sub}_{datafile}"

# datafile_data = allsubsdata_perFile[id]['data'] ## shape (electrode, time)
# datafile_scoring = allsubsdata_perFile[id]['scoring'] ## shape (time/sampling/30)
# datafile_data_cz = datafile_data[1,:]
# exmp_scoring_upsampled = yasa.hypno_upsample_to_data(datafile_scoring, 1/30, datafile_data_cz, sf_data=configu['sample_freq'], verbose=True)

# eng = matlab.engine.start_matlab()
# eng.SpindlesDetectionAlgorithm_fieldtrip(np.array(datafile_data_cz), configu['sample_freq'], np.array(exmp_scoring_upsampled))

# #spindles, spindlesOutEnergy, spindlesOutDuration, rej_cand = eng.SpindlesDetectionAlgorithm_fieldtrip(datafile_data_cz, configu['sample_freq'], exmp_scoring_upsampled)

# eng.quit()

Batch Nir&Andrillon Detection

In [25]:
def deleteAuxANFiles(andriNir_aux_output_dir):
    all_files_in_outputDir = os.listdir(andriNir_aux_output_dir)
    for file_in_outputDir in all_files_in_outputDir:
            os.remove(os.path.join( andriNir_aux_output_dir, file_in_outputDir ))


andriNir_code_dir = "C:\\Users\\User\\Cloud-Drive\\AnatArzi\\scripts\\Matlab_scripts - Sharons\\Andrillon_Nir_Algo"
andriNir_output_dir = f"{configu['outputs_dir_path']}\\{output_dir_name}\\Andrillon_Nir"
andriNir_aux_output_dir = f"{configu['outputs_dir_path']}\\{output_dir_name}\\Andrillon_Nir\\aux_mats"
spindles_output_columns =  ['spindleStartTime', 'spindleEndTime', 'peakTime', 'peakEnergy', 'peakEnergyNorm', 'freqSpindle', 'spindleDuration/SR', 'PowerSP', 'PowerAlpha', 'currentStage']
electrode_column_name = 'electrode'

if not os.path.exists(andriNir_output_dir):
    os.mkdir(andriNir_output_dir)

electrodes_names_eventDetect = ['F3','F4']#,'C3','C4','P3','P4','O1','O2']

all_electodes_ss_key = 'SS_algo-AN_all'

for id in allsubsdata_perFile:
    datafile_data = allsubsdata_perFile[id]['data'] ## shape (electrode, time)
    datafile_scoring = allsubsdata_perFile[id]['scoring'] ## shape (time/sampling/30)
    scoring_upsampled = yasa.hypno_upsample_to_data(datafile_scoring, 1/30, datafile_data, sf_data=configu['sample_freq'], verbose=True)
    ss_allElectrodes = pd.DataFrame()

    for electd_i, electrode_name_eventDetect in enumerate(electrodes_names_eventDetect):
        curr_electrode_num = np.where(configu['electrodes_names'] == electrode_name_eventDetect)[0][0]

        ## create aux files to use in MATLAB
        datafile_1elect_eeg = datafile_data[curr_electrode_num,:]
        if 2 not in datafile_scoring or 3 not in datafile_scoring:
            continue
        
        mat_to_save =  {'datafile_data': datafile_1elect_eeg, 'scoring_upsampled': scoring_upsampled, 'sample_freq': configu['sample_freq'], 'electrode_name':electrode_name_eventDetect}
        scipy.io.savemat(f"{andriNir_aux_output_dir}\\{allsubsdata_perFile[id]['subnum']}_{allsubsdata_perFile[id]['filenum']}_{electrode_name_eventDetect}AndScoring.mat",mat_to_save)

        ## run Andrillon & Nir SS detection over all subjects files
        eng = matlab.engine.start_matlab()
        eng.cd(andriNir_code_dir, nargout=0)
        out = eng.batch_useAndrillonNirEventDetection(andriNir_aux_output_dir, andriNir_output_dir,nargout=0)
        eng.quit()

        ## add the spindles data to the main subject dictionary
        try: 
            spindles_file_name = f"{andriNir_output_dir}\\Spindles_andrillonNir_{allsubsdata_perFile[id]['subnum']}_{allsubsdata_perFile[id]['filenum']}_{electrode_name_eventDetect}AndScoring.mat"
            matlabImport = scipy.io.loadmat(spindles_file_name, simplify_cells=True)
        except Exception: 
            print(f"Error importing spindles sub file at: {allsubsdata_perFile[id]['subnum']}_{allsubsdata_perFile[id]['filenum']}")
            continue

        ss = matlabImport['spindles']
        if np.size(ss) >0 :
            df = pd.DataFrame(np.double(ss))
            df.columns = spindles_output_columns
            tile_electrode = np.tile(f"{electrode_name_eventDetect}", ss.shape[0])[None].T
            df[electrode_column_name] = tile_electrode
            
            allsubsdata_perFile[id][f'SS_algo-AN@@{electrode_name_eventDetect}'] = df
            if electd_i ==0: ss_allElectrodes =  df
            else:  ss_allElectrodes = pd.concat([ss_allElectrodes,df])

        # sw = matlabImport['allWaves']
        # if np.size(sw) >0 :
        #     tile_electrode = np.tile(f"{electrode_name_eventDetect}", sw.shape[0])[None].T
        #     sw = np.hstack([sw, tile_electrode])
        #     allsubsdata_perFile[id][f'SW_algo-AN@@{electrode_name_eventDetect}'] = np.asarray(sw,dtype=object)  
        #     sw_allElectrodes = np.concatenate((sw_allElectrodes,sw))
 
    if not ss_allElectrodes.empty:
        allsubsdata_perFile[id][all_electodes_ss_key]  = ss_allElectrodes
        #allsubsdata_perFile[id][f'SW_algo-AN_all']  = sw_allElectrodes
    
deleteAuxANFiles(andriNir_aux_output_dir)

31-ינו-23 06:20:26 | WARNING | Hypnogram is SHORTER than data by 4.94 seconds. Padding hypnogram with last value to match data.size.
31-ינו-23 06:20:33 | WARNING | Hypnogram is SHORTER than data by 26.41 seconds. Padding hypnogram with last value to match data.size.
31-ינו-23 06:20:39 | WARNING | Hypnogram is SHORTER than data by 21.62 seconds. Padding hypnogram with last value to match data.size.
31-ינו-23 06:20:39 | WARNING | Hypnogram is SHORTER than data by 25.70 seconds. Padding hypnogram with last value to match data.size.
31-ינו-23 06:20:46 | WARNING | Hypnogram is SHORTER than data by 2.56 seconds. Padding hypnogram with last value to match data.size.
31-ינו-23 06:20:53 | WARNING | Hypnogram is SHORTER than data by 3.10 seconds. Padding hypnogram with last value to match data.size.
31-ינו-23 06:20:53 | WARNING | Hypnogram is SHORTER than data by 5.26 seconds. Padding hypnogram with last value to match data.size.
31-ינו-23 06:20:53 | WARNING | Hypnogram is SHORTER than data by 2

In [30]:
after_n2_exclution_key = f"{all_electodes_ss_key}_n2"
for id in allsubsdata_perFile:
    ss_allElectrodes = allsubsdata_perFile[id][all_electodes_ss_key]
    ss_allElectrodes_n2 = ss_allElectrodes.loc[ss_allElectrodes['currentStage'] == 2.0]
    if len(ss_allElectrodes_n2) == 0: 
        continue
    else:
        print(f"sub:{id}, before excludeNonN2Events:{len(ss_allElectrodes)}, after:{len(ss_allElectrodes_n2)}")
    allsubsdata_perFile[id][after_n2_exclution_key] = ss_allElectrodes_n2

sub:32_1, before excludeNonN2Events:621, after:273
sub:32_2, before excludeNonN2Events:824, after:312
sub:32_3, before excludeNonN2Events:824, after:312
sub:33_1, before excludeNonN2Events:1221, after:702
sub:33_2, before excludeNonN2Events:1417, after:1112
sub:33_3, before excludeNonN2Events:1417, after:1112
sub:33_4, before excludeNonN2Events:1417, after:1112
sub:34_1, before excludeNonN2Events:444, after:169
sub:34_2, before excludeNonN2Events:460, after:169
sub:34_3, before excludeNonN2Events:460, after:169
sub:35_1, before excludeNonN2Events:410, after:118
sub:35_3, before excludeNonN2Events:847, after:570
sub:35_4, before excludeNonN2Events:304, after:124
sub:38_1, before excludeNonN2Events:911, after:548
sub:38_2, before excludeNonN2Events:1023, after:598
sub:38_3, before excludeNonN2Events:757, after:545


In [ ]:
## create a spindle array, where each spindle occour once and attribued to the electrode where the spindle was the most powerful

ss_key_to_use = all_electodes_ss_key # after_n2_exclution_key
uniqeElctd_ss_key = f"{ss_key_to_use}_uniqeElctd"

def overlap(a, b):
    return a[1] >= b[0] and a[0] <= b[1]

for id in allsubsdata_perFile:
    if ss_key_to_use not in allsubsdata_perFile[id]:  continue
    filterd_events_allElectrodes = copy.copy(allsubsdata_perFile[id][ss_key_to_use])
    row_deleted = True
    k = 0
    while row_deleted:
        row_deleted = False
        for i in np.arange(k,np.shape(filterd_events_allElectrodes)[0]):
            i_ss_event = filterd_events_allElectrodes[i]
            i_ss_startTime = int(float(i_ss_event[0]))
            i_ss_endTime = int(float(i_ss_event[1]))
            i_ss_PowerSP = float(i_ss_event[7])
            for j in np.arange(i,np.shape(filterd_events_allElectrodes)[0]):
                if i == j: continue
                j_ss_event = filterd_events_allElectrodes[j]
                if overlap([i_ss_startTime,i_ss_endTime],[int(float(j_ss_event[0])),int(float(j_ss_event[1]))]):
                    row_deleted = True
                    if  i_ss_PowerSP >= float(j_ss_event[7]): # PowerSP
                        i_ss_event[-1] = np.append(i_ss_event[-1],j_ss_event[-1]) ## the electrode with the highr event power will be first
                        filterd_events_allElectrodes = np.delete(filterd_events_allElectrodes,j,axis=0)
                    else:
                        j_ss_event[-1] = np.append(j_ss_event[-1],i_ss_event[-1]) ## the electrode with the highr event power will be first
                        filterd_events_allElectrodes = np.delete(filterd_events_allElectrodes,i,axis=0)
                    break
            k+=1
            if row_deleted:
                break

    allsubsdata_perFile[id][uniqeElctd_ss_key] = filterd_events_allElectrodes            
    print(f"sub:{id}, before filt:{np.shape(allsubsdata_perFile[id][ss_key_to_use])[0]}, after:{np.shape(allsubsdata_perFile[id][uniqeElctd_ss_key])[0]}")

sub:32_1, before filt:2419, after:1695
sub:32_2, before filt:2231, after:1478
sub:32_3, before filt:0, after:0
sub:33_1, before filt:2491, after:1699
sub:33_2, before filt:3556, after:2265
sub:33_3, before filt:0, after:0
sub:33_4, before filt:0, after:0
sub:34_1, before filt:1363, after:1048
sub:34_2, before filt:1667, after:1227
sub:34_3, before filt:0, after:0
sub:35_1, before filt:1076, after:870
sub:35_3, before filt:1742, after:1176
sub:35_4, before filt:668, after:465
sub:38_1, before filt:2531, after:1647
sub:38_2, before filt:3091, after:1929
sub:38_3, before filt:2127, after:1372


In [ ]:
## create a spindle array, where each spindle occour once and attribued to the electrode where the spindle was the most powerful

ss_key_to_use = all_electodes_ss_key # after_n2_exclution_key
uniqeElctd_ss_key = f"{ss_key_to_use}_uniqeElctd"

def overlap(a, b):
    return a[1] >= b[0] and a[0] <= b[1]

for id in allsubsdata_perFile:
    if ss_key_to_use not in allsubsdata_perFile[id]:  continue
    filterd_events_allElectrodes = copy.copy(allsubsdata_perFile[id][ss_key_to_use])
    row_deleted = True
    while row_deleted:
        row_deleted = False
        for i,i_ss_event in enumerate(filterd_events_allElectrodes):
            i_ss_startTime = int(float(i_ss_event[0]))
            i_ss_endTime = int(float(i_ss_event[1]))
            i_ss_PowerSP = float(i_ss_event[7])
            for j,j_ss_event in enumerate(filterd_events_allElectrodes):
                if i == j: continue
                if overlap([i_ss_startTime,i_ss_endTime],[int(float(j_ss_event[0])),int(float(j_ss_event[1]))]):
                    row_deleted = True
                    if  i_ss_PowerSP >= float(j_ss_event[7]): # PowerSP
                        i_ss_event[-1] = np.append(i_ss_event[-1],j_ss_event[-1]) ## the electrode with the highr event power will be first
                        filterd_events_allElectrodes = np.delete(filterd_events_allElectrodes,j,axis=0)
                    else:
                        j_ss_event[-1] = np.append(j_ss_event[-1],i_ss_event[-1]) ## the electrode with the highr event power will be first
                        filterd_events_allElectrodes = np.delete(filterd_events_allElectrodes,i,axis=0)
                    break
            if row_deleted:
                break

    allsubsdata_perFile[id][uniqeElctd_ss_key] = filterd_events_allElectrodes            
    print(f"sub:{id}, before filt:{np.shape(allsubsdata_perFile[id][ss_key_to_use])[0]}, after:{np.shape(allsubsdata_perFile[id][uniqeElctd_ss_key])[0]}")

KeyboardInterrupt: 

In [31]:
## create a spindle array, where each spindle occour once and attribued to the electrode where the spindle was the most powerful

ss_key_to_use = after_n2_exclution_key # after_n2_exclution_key
uniqeElctd_ss_key = f"{ss_key_to_use}_uniqeElctd"

def overlap(a, b):
    return a[1] >= b[0] and a[0] <= b[1]

for id in allsubsdata_perFile:
    if ss_key_to_use not in allsubsdata_perFile[id]:  continue
    filterd_events_allElectrodes = copy.copy(allsubsdata_perFile[id][ss_key_to_use])
    filterd_events_allElectrodes = sorted(filterd_events_allElectrodes, key=lambda a_entry: float(a_entry[0]))  # sort by startTime

    row_deleted = True
    while row_deleted:
        row_deleted = False
        for i,i_ss_event in enumerate(filterd_events_allElectrodes):
            for j,j_ss_event in enumerate(filterd_events_allElectrodes):
                if i == j: continue

                i_ss_endTime = float(i_ss_event[1])
                j_ss_startTime = float(j_ss_event[0])
                if i_ss_endTime < j_ss_startTime: break

                i_ss_startTime = float(i_ss_event[0])
                j_ss_endTime = float(j_ss_event[1])
                i_ss_PowerSP = float(i_ss_event[7])
                j_ss_PowerSP = float(j_ss_event[7])
                if overlap([i_ss_startTime,i_ss_endTime],[j_ss_startTime,j_ss_endTime]):
                    row_deleted = True
                    if  i_ss_PowerSP >= j_ss_PowerSP: 
                        i_ss_event[-1] = np.append(i_ss_event[-1],j_ss_event[-1]) ## the electrode with the highr event power will be first
                        filterd_events_allElectrodes = np.delete(filterd_events_allElectrodes,j,axis=0)
                    else:
                        j_ss_event[-1] = np.append(j_ss_event[-1],i_ss_event[-1]) ## the electrode with the highr event power will be first
                        filterd_events_allElectrodes = np.delete(filterd_events_allElectrodes,i,axis=0)
                    break
            if row_deleted:
                break

    allsubsdata_perFile[id][uniqeElctd_ss_key] = filterd_events_allElectrodes            
    print(f"sub:{id}, before filt:{np.shape(allsubsdata_perFile[id][ss_key_to_use])[0]}, after:{np.shape(allsubsdata_perFile[id][uniqeElctd_ss_key])[0]}")

ValueError: could not convert string to float: 's'

In [46]:
## create a spindle array, where each spindle occour once and attribued to the electrode where the spindle was the most powerful

ss_key_to_use = after_n2_exclution_key # after_n2_exclution_key
uniqeElctd_ss_key = f"{ss_key_to_use}_uniqeElctd"

def overlap(a, b):
    return a[1] >= b[0] and a[0] <= b[1]

for id in allsubsdata_perFile:
    if ss_key_to_use not in allsubsdata_perFile[id]:  continue
    filterd_events_allElectrodes = copy.copy(allsubsdata_perFile[id][ss_key_to_use])
    filterd_events_allElectrodes = sorted(filterd_events_allElectrodes, key=lambda a_entry: a_entry[0])  # sort by startTime

    filtered = np.asarray([])
    deleted = filterd_events_allElectrodes = copy.copy(allsubsdata_perFile[id][ss_key_to_use])

    happened_together = np.asarray([])
    while np.shape(deleted)[0]>0:
        if np.size(happened_together) == 0:
            happened_together = np.append(happened_together, deleted[0])
            deleted = np.delete(deleted,0,0) # delete first row
        else:
            for i,i_ss_event in enumerate(happened_together):
                if overlap([i_ss_startTime,i_ss_endTime],[deleted[0],deleted[1]]):
                    happened_together = np.append(happened_together, deleted[0])
                    deleted = np.delete(deleted,0,0) # delete first row
                else:
                    break
        
        ## check max ps and add to filt
        happened_together = sorted(happened_together, key=lambda a_entry: float(a_entry[7]))  # sort by PowerSP
        max_electd_ss = happened_together[-1]
        max_electd_ss[-1] = happened_together[:,-1]
        filtered = np.append(max_electd_ss)
        happened_together = np.asarray([])

    allsubsdata_perFile[id][uniqeElctd_ss_key] = filtered   
    print(f"sub:{id}, before filt:{np.shape(allsubsdata_perFile[id][ss_key_to_use])[0]}, after:{np.shape(allsubsdata_perFile[id][uniqeElctd_ss_key])[0]}")

KeyError: 0

In [ ]:
## create a spindle array, where each spindle occour once and attribued to the electrode where the spindle was the most powerful

ss_key_to_use = after_n2_exclution_key # after_n2_exclution_key
uniqeElctd_ss_key = f"{ss_key_to_use}_uniqeElctd"

def overlap(a, b):
    return a[1] >= b[0] and a[0] <= b[1]

for id in allsubsdata_perFile:
    if ss_key_to_use not in allsubsdata_perFile[id]:  continue
    filterd_events_allElectrodes = allsubsdata_perFile[id][ss_key_to_use].copy(deep=True)
    
    filterd_events_allElectrodes.sort_values(by=['spindleStartTime'],inplace=True) 
    deleted = filterd_events_allElectrodes.copy(deep=True)
    simultan = pd.DataFrame([])
    while len(deleted)>0:
        if len(simultan)==0:
            simultan = pd.concat([simultan, deleted.iloc[0]])
            deleted.drop([0], axis=0, inplace=True)
        else:
            simultan = simultan.reset_index()  # make sure indexes pair with number of rows
            for index, simultan_row in simultan.iterrows():
                print(row['c1'], row['c2'])
                simultan_0 = simultan_row['spindleStartTime']
                simultan_1 = simultan_row['spindleEndTime']
                deleted_0 = deleted.iloc[0]['spindleStartTime']
                deleted_1 = deleted.iloc[0]['spindleEndTime']
                if overlap([simultan_0,simultan_1],[deleted_0,deleted_1]):
                    simultan = pd.concat([simultan, deleted.iloc[0]])
                    deleted.drop([0], axis=0, inplace=True)
                else:
                    break
                
        ############3 Im super tired continue from here
        ## check max ps and add to filt
        simultan.sort_values(by=['PowerSP'],inplace=True) 
        max_electd_ss = simultan.iloc[0]
        max_electd_ss[-1] = simultan[:,-1]
        filtered = np.append(max_electd_ss)
        simultan = np.asarray([])

    allsubsdata_perFile[id][uniqeElctd_ss_key] = filtered   
    print(f"sub:{id}, before filt:{np.shape(allsubsdata_perFile[id][ss_key_to_use])[0]}, after:{np.shape(allsubsdata_perFile[id][uniqeElctd_ss_key])[0]}")

In [59]:
filterd_events_allElectrodes.iloc[0]

spindleStartTime      1725777.0
spindleEndTime        1726208.0
peakTime              1725958.0
peakEnergy            26.955765
peakEnergyNorm         9.235046
freqSpindle           13.972056
spindleDuration/SR        0.864
PowerSP               10.013066
PowerAlpha             7.012449
currentStage                2.0
electrode                    F3
Name: 61, dtype: object

Change scoring to fit EDF_viewer events format

In [ ]:
def format_scoring_to_edfViewer(score):
    if score == 0:
        return 'W'
    elif score ==1:
        return 'N1'
    elif score ==2:
        return 'N2'
    elif score ==3:
        return 'N3'
    elif score ==4:
        return 'TREM'
    elif score ==5:
        return 'PREM'
    elif score ==6:
        return 'MOVE'
    elif score ==7:
        return 'ARTIFACT'
    else:
        Exception('no such score')
        

for id in allsubsdata_perFile:
    curr_file_scoring = allsubsdata_perFile[id]['scoring']
    new_format_score = np.zeros((len(curr_file_scoring),3), dtype=object)
    for ind, score in enumerate(curr_file_scoring):
            new_format_score[ind,:] = [30*ind,30,format_scoring_to_edfViewer(score)] ## onset (sec), duration, desc

    allsubsdata_perFile[id]['scoring_efdViewFormat'] = new_format_score
    # np.savetxt('test2.txt', new_format_score, delimiter='\t',fmt='%s')

Change spindles events to fit EDF_viewer events format

In [ ]:
# [spindleStartTime spindleEndTime peakTime peakEnergy peakEnergyNorm freqSpindle spindleDuration/SR PowerSP PowerAlpha currentStage];
def EDFViewerformat(key_name):
        curr_file_ss = allsubsdata_perFile[id][key_name]
        new_format_ss = np.zeros((np.shape(curr_file_ss)[0],3), dtype=object)
        for ind, ss in enumerate(new_format_ss):
                SpindleStartTime = np.double(curr_file_ss[ind,0]) / np.double(configu['sample_freq'])  
                SpindleDuration = (np.double(curr_file_ss[ind,1])  -np.double(curr_file_ss[ind,0])) / np.double(configu['sample_freq'])
                if np.size(curr_file_ss[ind,10]) > 1:  
                        event_desc = f"{key_name.split('_')[0]}@@{curr_file_ss[ind,10][0]}"
                else:
                        event_desc = f"{key_name.split('_')[0]}@@{curr_file_ss[ind,10]}"
                new_format_ss[ind,:] = [SpindleStartTime,SpindleDuration,event_desc] ## onset (sec), duration,desc
        allsubsdata_perFile[id][f'{key_name}_efdViewFormat'] = new_format_ss



events_types_for_save = [uniqeElctd_ss_key]
for event_type in events_types_for_save: 
        for id in allsubsdata_perFile:
                if event_type not in allsubsdata_perFile[id]:
                        print(f"no {event_type} for sub {id}")
                        continue
                EDFViewerformat(event_type)

In [ ]:
edfViewFormat_events_output_dir = f'{fig_output_dir}\\EDFViewFormat_events'
if not os.path.exists(edfViewFormat_events_output_dir):
    os.mkdir(edfViewFormat_events_output_dir)

events_types_for_save = ['scoring',uniqeElctd_ss_key]
# Save events (scoring and ss) per file, in a format suitable for EDF_viewer
for id in allsubsdata_perFile:
        filename = f"{allsubsdata_perFile[id]['subnum']}_{allsubsdata_perFile[id]['filenum']}_events"
        all_events_with_header = np.asarray([['Onset',"Duration","Annotation"]],dtype=object)
        for event_type in events_types_for_save:
                events_type_found = [ v for k, v in allsubsdata_perFile[id].items() if (event_type in k) and ('_efdViewFormat' in k)]
                if np.size(events_type_found) == 0: continue
                for events_found in events_type_found:
                        for event_found in events_found:
                                all_events_with_header = np.concatenate((all_events_with_header,[event_found]),dtype=object)
        np.savetxt(f"{edfViewFormat_events_output_dir}\\{filename}.txt", all_events_with_header, delimiter='\t',fmt='%s')

Test TP / FP for the algorithm of Andrillon-Nir

In [ ]:
## get the array of before manual scanning
txt_file_name = "C:\\Users\\User\\Cloud-Drive\\BigFiles\\OmissionExpOutput\\eventDetection\\done_on_preProcessed\\EDFViewFormat_events\\32_2_events.txt"
before = np.loadtxt(txt_file_name, delimiter="\t",dtype='object')
print(np.shape(before))
before[1:,0] = [np.double(x) for x in before[1:,0]]
before = np.delete(before,0,0)
ss_ind = np.array([],dtype=int)
for ind_i, desc in enumerate(before[:,2]):
    if "SS" in desc:
        ss_ind = np.append(ss_ind, ind_i)
before_ss = before[ss_ind,:]
print(np.shape(before_ss))

## get the array of after manual scanning
txt_file_name = "C:\\Users\\User\\Cloud-Drive\\BigFiles\\OmissionExpOutput\\eventDetection\\done_on_preProcessed\\EDFViewFormat_events\\s_32_sleep2_preProcessed_annotations.txt"
after = np.loadtxt(txt_file_name, delimiter="\t",dtype='object')
print(np.shape(after))
after[1:,0] = [np.double(x) for x in after[1:,0]]

ss_ind = np.array([],dtype=int)
for ind_i, desc in enumerate(after[:,2]):
    if "SS" in desc:
        ss_ind = np.append(ss_ind, ind_i)
after_ss = after[ss_ind,:]
print(np.shape(after_ss))

## compare to find rate od TP and FP
TP = 0
FP = 0
for ss_auto in before_ss:
    found = False
    for ss_manu in after_ss:
        if ss_auto[0] == ss_manu[0]:
            TP +=1
            found = True
            break
    if found == False:
        FP +=1

print(TP)
print(FP)

(510, 3)
(204, 3)
(737, 3)
(235, 3)
156
48
